In [4]:
%run "../01-check_setup.ipynb"

SAP HANA Client for Python: 2.23.25021400
Connected to SAP HANA db version 4.00.000.00.1764669230 (fa/CE2025.28) 
at c5889dd5-e0f6-4930-8408-94d53ca61dbf.hna0.prod-us10.hanacloud.ondemand.com:443 as CODEJAMHANAAI00
Current time on the SAP HANA server: 2025-12-12 20:29:57.636000


In [1]:
import requests, os

os.environ["USER_AGENT"] = f"{requests.get('https://httpbingo.org/user-agent').json()['user-agent']}"
os.environ["USER_AGENT"]

'python-requests/2.32.5'

In [8]:
import zipfile
import requests
from io import BytesIO

# URL of the zipped file
zip_url = "https://docs.python.org/3.13/archives/python-3.13-docs-text.zip"

# Download the zipped file
response = requests.get(zip_url)
zip_file = BytesIO(response.content)


In [9]:

# Load documents from the zipped file
documents_from_zip = []
with zipfile.ZipFile(zip_file, 'r') as z:
    for file_name in z.namelist():
        if not z.getinfo(file_name).is_dir():
            with z.open(file_name) as f:
                documents_from_zip.append({'metadata': {'file_name': file_name}, 'content': f.read().decode('utf-8')})

# Check the number of documents loaded
print(f"Number of documents loaded: {len(documents_from_zip)}")

Number of documents loaded: 512


In [10]:
## Select only records where file_name contains '/faq/'
faq_documents = [doc for doc in documents_from_zip if '/faq/' in doc['metadata']['file_name']]  
print(f"Number of FAQ documents: {len(faq_documents)}")

Number of FAQ documents: 9


In [11]:
print(faq_documents[2]['content'])

Library and Extension FAQ
*************************


General Library Questions


How do I find a module or application to perform task X?
--------------------------------------------------------

Check the Library Reference to see if there's a relevant standard
library module.  (Eventually you'll learn what's in the standard
library and will be able to skip this step.)

For third-party packages, search the Python Package Index or try
Google or another web search engine.  Searching for "Python" plus a
keyword or two for your topic of interest will usually find something
helpful.


Where is the math.py (socket.py, regex.py, etc.) source file?
-------------------------------------------------------------

If you can't find a source file for a module it may be a built-in or
dynamically loaded module implemented in C, C++ or other compiled
language. In this case you may not have the source file or it may be
something like "mathmodule.c", somewhere in a C source directory (not
on the Python

In [12]:
import pandas as pd

## Convert faq_documents to a DataFrame `df_faq`
df_faq = pd.DataFrame(faq_documents)


In [13]:
import markdown
from IPython.display import HTML

# Convert Markdown to HTML
html = markdown.markdown(df_faq['content'][0])

display(HTML(html))


In [14]:
# Convert column metadata of df_faq to string
df_faq['metadata'] = df_faq['metadata'].astype(str)
df_faq['content_html'] = df_faq['content'].apply(markdown.markdown)

In [15]:
display(df_faq.dtypes)
df_faq=df_faq.convert_dtypes()
display(df_faq.dtypes)

metadata        object
content         object
content_html    object
dtype: object

metadata        string[python]
content         string[python]
content_html    string[python]
dtype: object

In [16]:
hana_table_name="FAQ"

In [17]:
## Create a table in SAP HANA from `df_faq`
from hana_ml.dataframe import create_dataframe_from_pandas
hdf_faq_bronze = create_dataframe_from_pandas(connection_context=myconn, 
                                              pandas_df=df_faq, 
                                              table_name=hana_table_name, 
                                              force=True,
                                              object_type_as_bin=True,
                                              table_structure={'metadata': 'NVARCHAR(5000)' ,
                                                               'content': 'NCLOB', 'content_html': 'NCLOB'}
                                              )

100%|██████████| 1/1 [00:00<00:00, 50.25it/s]


In [18]:
import pandas as pd
pd.set_option('max_colwidth', 256) 

hdf_faq_bronze.head().collect().T

,0
metadata,{'file_name': 'python-3.13-docs-text/faq/installed.txt'}
content,"""Why is Python Installed on my Computer?"" FAQ\n*********************************************\n\n\nWhat is Python?\n===============\n\nPython is a programming language. It's used for many different\napplications. It's used in some high schools and coll..."
content_html,"<p>""Why is Python Installed on my Computer?"" FAQ</p>\n<hr />\n<h1>What is Python?</h1>\n<p>Python is a programming language. It's used for many different\napplications. It's used in some high schools and colleges as an\nintroductory programming langua..."
